In [32]:
import geemap, ee
import geopandas as gpd
import json
import numpy as np

In [ ]:
import sys
print(sys.version)

In [2]:
import ee 
import geemap.eefolium as geemap

In [33]:
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

In [35]:
# helper function to put the geodataframe in the right format for constructing an ee object
def shp_to_ee_fmt(city, level):
    dic = {'Satkhira':{2:56,3:469,4:4307},'Sirajganj':{2:59,3:496,4:4493}}
    ind = dic[city][level]
    shp_path = "/Users/MT/Nextcloud/Projects/GOV-BGD20GIZ7333_CRISC_AI/Analysis/bgd_adm_bbs_20201113_SHP/bgd_admbnda_adm"+str(level)+"_bbs_20201113.shp"
    gdf = gpd.read_file(shp_path, crs='EPSG:4326')
    x = gdf.iloc[ind:ind+1,:]
    data = json.loads(x.to_json())
    return data['features'][0]['geometry']['coordinates']

# Source shp: https://data.humdata.org/dataset/administrative-boundaries-of-bangladesh-as-of-2015

In [36]:
viirs = ee.Image(ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2019-01-01","2019-12-31").median().select('avg_rad'))

In [6]:
srtm = ee.Image("CGIAR/SRTM90_V4").select('elevation')
srtmVis = {'min':-1, 'max':14,'palette':['#f7fcfd','#e5f5f9','#ccece6','#99d8c9','#66c2a4','#41ae76','#238b45','#005824']}

In [7]:
water = ee.Image("JRC/GSW1_3/GlobalSurfaceWater").select('occurrence')
waterVis= {"min":0, "max":50,"palette":['lightblue', 'blue'], "bands":"occurrence"}

In [8]:
ghslSet = ee.ImageCollection('JRC/GHSL/P2016/SMOD_POP_GLOBE_V1').filter(ee.Filter.date('2015-01-01', '2015-12-31')).select('smod_code').median();
ghslSetVis= {"min":0.0, "max":3.0,"palette":['000000', '448564', '70daa4', 'ffffff']}

In [9]:
ghslPop = ee.ImageCollection("JRC/GHSL/P2016/POP_GPW_GLOBE_V1").filter(ee.Filter.date('2015-01-01', '2015-12-31')).select('population_count').median();
ghslPopVis= {"min":0.0, "max":5000.0,"palette":['000000', '448564', '70daa4', 'ffffff']}

In [37]:
# Get Sentinal
aoi_bd = ee.Feature(ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Bangladesh')).first()).geometry()


first = (ee.ImageCollection('COPERNICUS/S2_SR')
         .filterBounds(aoi_bd)
         .filterDate('2019-01-01', '2019-12-31')
         .sort('CLOUDY_PIXEL_PERCENTAGE')
         .first())

In [38]:
#AOI: Satkhira District (Khulna Division)
aoi = ee.Geometry.MultiPolygon(shp_to_ee_fmt(city = 'Satkhira', level = 3))

# initialize the map
map1 = geemap.Map()
map1.centerObject(aoi, 13)
#map1.addLayer(viirs.clip(aoi), {}, "VIIRS-DNB Nightlights", opacity=0.5)
#map1.addLayer(ghslSet.clip(aoi), ghslSetVis, 'GHSL Degree of Urbanization', opacity=0.5)
#map1.addLayer(ghslPop.clip(aoi), ghslPopVis, 'GHSL Population', opacity=0.5)
#map1.addLayer(srtm.clip(aoi), {'min':-1, 'max':14}, 'SRTM Elevation', opacity = 0.6)
#map1.addLayer(water.clip(aoi), waterVis, 'JRC Water Prevalence', opacity = 0.6)
map1.addLayer(
    first.clip(aoi), {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000}, 'first')
map1.addLayerControl()
map1

EEException: An internal error has occurred (request: c0a945cb-133f-474b-9ecf-b12662afc4c8)

In [19]:
import datapane as dp
dp.login(token='2b8de896fc86c1d7b568474c553dca773dc3c606')

Connected successfully to https://datapane.com as mtyrrell


'mtyrrell'

## Layers
* Google maps (comparison with OSM)
* VIIRS Nightlights Average Radiance 2019 median (approx. 500m resolution; B&W scale from dark to light indicating higher luminance)
* GHSL Settlements Classification 2015 (1Km resolution)
    - Black: Inhabited areas
    - Dark Green: RUR (rural grid cells)
    - Light Green: LDC (low density clusters)
    - White: HDC (high density clusters)
*	GHSL Population Estimations 2015 (250m resolution with scale from dark to light colours indicating higher population)
*	SRTM Elevation (90m resolution; B&W scale from dark to light indicating higher elevation values)
*	JRC Water surface levels: Occurrence 1984-1999, 2000-2020 (30m resolution; scale from light to dark blue indicating higher water prevalence; no colour = water never detected)


In [20]:
map1.publish(name='Satkhira Municipal: Multi Layer', description='SatKhira GAUL Level 3 with VIIRS, GHSL, Topo and Water', visibility='PUBLIC')

Publishing document and associated data - *please wait...*

Your report doesn't contain any text - did you know you can add text to your report from your browser once published?

Your report only contains a single element - did you know you can add multiple plots and tables to a report, add text to it and export directly to Medium once published?

Report successfully published at https://datapane.com/u/mtyrrell/reports/satkhira-municipal-multi-layer/ - you can edit and add additional text online

In [26]:
#AOI: Sirajganj District (Rajshahi Division)
aoi = ee.Geometry.MultiPolygon(shp_to_ee_fmt(city = 'Sirajganj', level = 3))

# initialize the map
map2 = geemap.Map()
map2.centerObject(aoi, 12)
map2.addLayer(viirs.clip(aoi), {}, "VIIRS-DNB Nightlights", opacity=0.5)
map2.addLayer(ghslSet.clip(aoi), ghslSetVis, 'GHSL Degree of Urbanization', opacity=0.5)
map2.addLayer(ghslPop.clip(aoi), ghslPopVis, 'GHSL Population', opacity=0.5)
map2.addLayer(srtm.clip(aoi), {'min':5, 'max':25}, 'SRTM Elevation', opacity = 0.6)
map2.addLayer(water.clip(aoi), waterVis, 'JRC Water Prevalence', opacity = 0.6)
map2.addLayerControl()
map2

In [27]:
map2.publish(name='Sirajganj Municipal: Multi Layer', description='Sirajganj GAUL Level 3 with VIIRS, GHSL, Topo and Water', visibility='PUBLIC')

Publishing document and associated data - *please wait...*

Your report doesn't contain any text - did you know you can add text to your report from your browser once published?

Your report only contains a single element - did you know you can add multiple plots and tables to a report, add text to it and export directly to Medium once published?

Report successfully published at https://datapane.com/u/mtyrrell/reports/sirajganj-municipal-multi-layer/ - you can edit and add additional text online